#### SBERT
- BERT 모델 : 문장 이해용 Encoder
    - 문장 쌍 비교
- SBERT 모델 : 문장 의미 임베딩 
    - 벡터의 비교용 

In [ ]:
# 라이브러리 설치 
# !pip install sentence-transformers

In [ ]:
import torch 
from sentence_transformers import SentenceTransformer, util

In [ ]:
# 모델을 로드 -> 두개의 문장을 비교(코사인 유사도)
# 다목적 한국 SBERT
model_name = 'jhgan/ko-sroberta-multitask'
# 문장 유사도 특화 
model_name2 = 'BM-K/KoSimCSE-roberta-multitask'

sbert = SentenceTransformer(model_name)
sbert2 = SentenceTransformer(model_name2)

In [ ]:
# 최대 토큰의 길이를 설정 
sbert.max_seq_length = 256
sbert2.max_seq_length = 256

In [ ]:
# 두개의 문장을 비교 
doc1 = "이 카메라는 색감이 자연스럽고 배터리도 오래간다"
doc2 = "배터리 성능이 좋고 사진 품질이 뛰어나다"

In [ ]:
# 두개의 문장을 임베딩 -> 코사인 유사도 계산 
# sbert인 경우 
with torch.inference_mode():
    emb1 = sbert.encode(doc1, convert_to_tensor=True, normalize_embeddings=True)
    emb2 = sbert.encode(doc2, convert_to_tensor=True, normalize_embeddings=True)
# 코사인 유사도 계산
cos_sim = util.cos_sim(emb1, emb2).item()
print("유사도 : ", round(cos_sim, 4))

In [ ]:
# 두개의 문장을 임베딩 -> 코사인 유사도 계산 
# sbert2인 경우 
with torch.inference_mode():
    emb1 = sbert2.encode(doc1, convert_to_tensor=True, normalize_embeddings=True)
    emb2 = sbert2.encode(doc2, convert_to_tensor=True, normalize_embeddings=True)
# 코사인 유사도 계산
cos_sim = util.cos_sim(emb1, emb2).item()
print("유사도 : ", round(cos_sim, 4))

In [ ]:
emb1.shape

In [ ]:
sentences = [
    '삼성전자 주가가 올랐다', 
    "코스피가 상승 마감했다", 
    '비가 많이 와서 항공편이 취소됬다'
]

with torch.inference_mode():
    embs = sbert2.encode(sentences, convert_to_tensor=True, 
                         normalize_embeddings=True)

# 3개의 문장에서의 유사도를 확인 
sim_metrix = util.cos_sim(embs, embs)
print(sim_metrix)

In [ ]:
new_sentence = "증시가 강세였다"
# 임베딩 
new_emb = sbert2.encode(new_sentence, convert_to_tensor=True, 
                        normalize_embeddings=True)
# 유사도가 높은 상위의 n개 확인 
top_n = 2
hits = torch.topk(
    util.cos_sim(new_emb, embs).squeeze(0), k = top_n
)
hits

In [ ]:
for score, idx in zip( hits.values.tolist(), hits.indices.tolist() ):
    print(f"{sentences[idx]} | score : {round(score, 3)}")

### 연습 
- ratings_train.txt 파일을 로드 
- 결측치 제거 
- document 컬럼의 문자 정규화(특수문자 제거, 2칸 이상의 공백 제거, 좌우 공백 제거) 
- 중복 document 제거 , 글자의 수가 1개 이하인 행은 제거  
- DataFrame에서 sample(n = 10000, random_state=42)로 임의의 데이터를 추출하여 저장 (head() -> 상위 데이터 | tail() -> 하위 데이터 | sample() -> 무작위 데이터) 
- train, test 셋으로 8:2 로 데이터분할
- sbert 모델은 'BM-K/KoSimCSE-roberta-multitask'을 이용
- Dataset을 정의 (Trainer 이용하지 않고 Dataset과 DataLoader 사용)
    - `__init__(self, document, labels)`
        - 입력받은 document와 labels를 document는 SBERT 모델을 이용하여 인코딩 
        - labels 데이터를 tensor형태로 변환 
    - `__len__(self)` 함수는 라벨의 길이를 되돌려준다
    - `__getitem__(self, idx)` 함수는 인코딩된 데이터[idx], label[idx]를 되돌려준다
- Dataset를 train, test를 이용해서 Dataset을 생성 
- DataLoarder를 이용하여 배치의 사이즈는 128 shuffle은 True 구성한다. 

In [1]:
import re
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sentence_transformers import SentenceTransformer

c:\Users\ekfla\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 데이터 로드 
df = pd.read_csv("../data/ratings_train.txt", sep='\t')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [3]:
# 결측치를 제거 
df.dropna(subset='document', inplace=True)

In [4]:
def normalize(text):
    text = re.sub(r'[^가-힣0-9a-zA-Z\s\.]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [5]:
df['document'] = df['document'].map(normalize)

In [6]:
# 중복 데이터를 제거 
df.drop_duplicates(subset='document', inplace=True)

In [7]:
len(df)

144734

In [8]:
# 문자열의 길이가 1 이하는 제거 -> 1초과인 데이터만 확인 
flag = df['document'].str.len() > 1
df = df.loc[flag,]

In [9]:
# 랜덤한 데이터 10000개추출 sample()
df = df.sample(n = 10000, random_state=42).reset_index(drop=True)

In [10]:
# train, test 분할 (8:2)
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['label']
)

In [11]:
train_df['label'].value_counts()

label
0    4006
1    3994
Name: count, dtype: int64

In [12]:
model_name3 = 'BM-K/KoSimCSE-roberta-multitask'
sbert3 = SentenceTransformer(model_name3)

No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.


In [13]:
# Dataset 정의 
class SBERTDataset(Dataset):
    # 생성자 함수 -> document, labels 받아와서 document 임베딩, labels는 tensor화
    def __init__(self, document, labels):
        # no_grad() -> 자동 미분 일시 정지 
        # inference_mode() -> 추론 모드 
        with torch.inference_mode():
            # 로드한 모델을 이용해서 encode 작업 
            # convet_to_tensor -> 결과값을 tensor로 받을것인가? (False : list)
            # normalize_embeddings -> L2 정규화 할것인가?
            self.emb = sbert3.encode(
                document, convert_to_tensor = True, normalize_embeddings=True
            )
        # labels를 tensor화 
        self.labels = torch.tensor(labels, dtype=torch.long)
    def __len__(self):
        # labels의 길이를 되돌려준다.
        return len(self.labels)
    def __getitem__(self, idx):
        return self.emb[idx], self.labels[idx]

In [14]:
# Dataset의 형태로 데이터프레임을 변환 
train_ds = SBERTDataset( train_df['document'].tolist(), train_df['label'].tolist() )
test_ds = SBERTDataset(test_df['document'].tolist(), test_df['label'].tolist())

In [15]:
# DataLoader를 이용해서 배치 사이즈만큼의 데이터를 생성 
train_dl = DataLoader(train_ds, batch_size = 128, shuffle = True)
test_dl = DataLoader(test_ds, batch_size = 128, shuffle=True)

In [ ]:
# 분류 모델 정의 ( 입력받은 데이터 -> document tensor, label tensor )
class MLPHead(nn.Module):
    # 선형 모델에 데이터를 입력하는 형태 
    # 선형 모델을 정의할때 인자값 (Linear( 입력데이터의 피쳐의 수, 출력의 피쳐의 수 ))
    def __init__(self, input_dim, hidden = 256, num_classes = 2):
        super().__init__()
        # 다중 퍼셉트론층 구성 
        self.net = nn.Sequential(
            # 선형 모델 
            nn.Linear( input_dim, hidden ), # 입력 768 차원에서 출력은 256 차원
            nn.ReLU(),                      # 비선형의 구조를 이해하기 위한 작업
            nn.Dropout(0.2),                # 과적합 방지를 위한 소실 작업
            nn.Linear( hidden, num_classes )
        )
    # 순전파 함수 -> 독립변수를 받아서 예측 값을 되돌려준다. 
    def forward(self, x):
        # x : 독립 변수 (document 데이터를 임베딩하고 배치로 묶은 데이터)
        result = self.net(x) # 출력이 2차원인 확률 데이터들
        return result


In [18]:
# MLPHead class를 생성하기 위해서는 input_dim 매개변수에 인자는 필수 항목
# imput_dim -> 입력 데이터(독립변수)의 피쳐의 수를 의미
# 입력데이터 -> sbert3모델에서 임베딩이 된 독립변수의 피쳐의 수
# sbert3에서 설정이 된 출력 피쳐의 수를 변수에 저장 
in_dim = sbert3.get_sentence_embedding_dimension() # 출력 피쳐의 수를 되돌려주는 내장 함수 
in_dim

768

In [19]:
# MLPHead 모델을 생성 
clf = MLPHead(in_dim)
# 손실함수 -> 예측값과 실젯값의 차이를 확인하는 함수 
crit = nn.CrossEntropyLoss()
# 옵티마이저 
opt = torch.optim.AdamW(clf.parameters(), lr=2e-4)

In [ ]:
# 학습 루프 
# 학습 모드 전환 
clf.train()

for epoch in range(5):
    total = 0.0
    for x, y in train_dl:
        # x : document데이터가 임베딩 벡터가 된 묶음(tensor)
        # y : labels데이터가 tensor형태 묶음
        opt.zero_grad()
        # 순전파
        logits = clf(x)
        # 손실 계산 (예측값, 실젯값)
        loss = crit( logits, y )
        # 역전파
        loss.backward()
        # 스탭 
        opt.step()
        total += loss.item() * x.size(0)
    print(f"epoch : {epoch}, loss : {total/len(train_ds)}")

epoch : 0, loss : 0.6295463409423828
epoch : 1, loss : 0.4925076322555542
epoch : 2, loss : 0.43932614731788633
epoch : 3, loss : 0.4204492735862732
epoch : 4, loss : 0.4112177505493164


In [22]:
# 테스트 데이터를 이용하여 정확도, f1_score 확인
clf.eval()

y_true, y_pred = [], []

with torch.inference_mode():
    for x, y in test_dl:
        logits = clf(x)  # 예측 데이터 -> [[ 0.xxx, 0.xxxx ], [], [] , ...]
        pred = logits.argmax(dim=1).tolist()    # 예측 데이터 -> [0, 1, 1, 0, ...]
        # y_true에 y를 리스트의 형태로 변환하고 데이터를 확장시킨다. 
        y_true.extend(y.tolist())
        y_pred += pred
print(y_true)
print(y_pred)

[1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 

In [23]:
print('accuracy_score', accuracy_score(y_true, y_pred))
print('f1_score', f1_score(y_true, y_pred))

accuracy_score 0.814
f1_score 0.8119312436804853


In [29]:
samples = [
    '와... 영화 진짜 최고였습니다. 또 보고 싶습니다.', 
    '스토리가 엉망이고 연기도 별로였다. 추천할만한 영화는 아니다', 
    '그럭저럭 볼만했지만 크게 인상적이진 않았다.'
]

id2label = {
    0 : '부정', 
    1 : '긍정'
}
# 예측값을 되돌려주는 함수
# model(독립변수)
# 독립변수의 튜닝 -> 
@torch.no_grad()
def predict_review(
    texts, 
    batch_size = 128
):
    # texts : 예측하려고 하는 리뷰의 원문 데이터들
    # batch_size : 묶음의 크기 

    # texts의 정규화 -> texts(list형태) -> map(), for문을 이용하여 정규화 
                #   -> texts(str) -> 1. 문자열을 정규화 함수에 입력, 
                # 2. 문자열이면 리스트의 형태로 변환
    if isinstance(texts, str):
        texts = [texts]
    
    # 정규화 함수에 리뷰 데이터를 넣어준다. 
    texts_norm = [normalize(t) for t in texts]

    # 2개의 모델을 평가모드 전환 clf, sbert3
    sbert3.eval()
    clf.eval()

    # 결과 값
    result = []

    # 배치 데이터로 구성 -> encode -> 분류 모델에 데이터 입력 -> 출력 값을 설정 -> result에 대입
    for idx in range(0, len(texts_norm), batch_size):
        batch_texts = texts_norm[ idx : idx + batch_size ]
        # texts = ['a', b', 'c'] 
        # batch_size = 2
        # 첫번째 반복 구간에서는 
        # batch_texts -> ['a', 'b']
        # embs -> 벡터화 -> 열의 개수는 sbert3의 아웃풋의 차원의 수(768) 
        #               -> 행의 개수는 len(batch_texts)
        # probs -> [ [ 0.3, 0.7 ] , [ 0.51, 0.49 ]]
        # preds -> [ 1 , 0 ]
        # sbert3의 encode함수를 이용하여 임베딩 벡터 생성 
        embs = sbert3.encode(
            batch_texts, 
            convert_to_tensor=True, 
            normalize_embeddings=True
        )
        # embs를 clf 모델을 이용하여 예측 확률 데이터를 생성 
        logits = clf(embs)
        probs = logits.softmax(dim = -1)
        preds = probs.argmax(dim=-1).tolist()

        for idx2, pred in enumerate(preds):
            # 첫번째 반복문의 1번 루프에서 preds -> [1, 0]
            # 두번째 반복문의 첫번째 루프 
            # idx2 -> 0
            # pred -> 1
            # prob -> prods[0, 1] -> 0.7
            # review -> texts[ 0 + 0 ] -> texts[0]
            # idx2 : 인덱스 
            # pred : 예측 값(예측 확률의 인덱스 - 확률이 높은 곳의 인덱스(0,1))
            # 높은 예측율
            prob = float( probs[idx2, pred] )
            # 리뷰의 원문 
            # 첫번째 반복문의 반복 횟수? -> len(texts) / batch_size + 1
            # 두번째 반복문의 반복 횟수? -> 
            # idx -> 배치의 시작지점
            # idx2 -> 시작점부터 얼마만큼 이동했는가?
            review = texts[idx + idx2]
            # 긍정/부정 라벨링
            label = id2label[pred]
            # prob, review, label들을 result에 추가 
            result.append(
                {
                    'text' : review, 
                    'prob' : prob, 
                    'label' : label
                }
            )
    return result



In [30]:
out_data = predict_review(samples)

In [31]:
out_data

[{'text': '와... 영화 진짜 최고였습니다. 또 보고 싶습니다.',
  'prob': 0.9746094942092896,
  'label': '긍정'},
 {'text': '스토리가 엉망이고 연기도 별로였다. 추천할만한 영화는 아니다',
  'prob': 0.9662363529205322,
  'label': '부정'},
 {'text': '그럭저럭 볼만했지만 크게 인상적이진 않았다.',
  'prob': 0.6602864861488342,
  'label': '부정'}]